In [7]:
import json
from langdetect import detect
from num2words import num2words
from googletrans import Translator
from tqdm.autonotebook import tqdm
from ofa.ofa_infer import OFAInference
from translator import EnViVinAITranslator, JaEnTranslator

In [2]:
ofa = OFAInference()

/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


In [8]:
envi_translator = EnViVinAITranslator()
jaem_translator = JaEnTranslator()

In [9]:
gg_translator = Translator()

In [4]:
envi_translator.translate_vi2en("có bao nhiêu người đang đứng bán phở?")

'How many people are standing there selling noodles?'

In [5]:
with open('data/test/official_evjvqa_public_test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
    
test_images = test_data['images']
test_annotations = test_data['annotations']

len(test_images), len(test_annotations)

(558, 5015)

In [5]:
# img_path ='./data/test/public-test-images'

# imgid2imgpath = {}
# lang_count = {}

# for i in range(len(test_images)):
#     timg = test_images[i]
#     imgid2imgpath[timg['id']] = f"{img_path}/{timg['filename']}"
    
# for i in range(len(test_annotations)):
#     anno = test_annotations[i]
#     anno['img_path'] = imgid2imgpath[anno['image_id']]
#     lang = detect(anno['question'])
#     anno['language'] = lang
#     if lang in lang_count:
#         lang_count[lang] += 1
#     else:
#         lang_count[lang] = 1

In [6]:
# lang_count

In [7]:
# with open('./data/test/official_evjvqa_public_test_lang.json', 'w', encoding='utf=8') as f:
#     json.dump(test_data, f, indent=4, ensure_ascii=False)

## RUNNING

In [6]:
with open('data/test/official_evjvqa_public_test_lang.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
    
test_images = test_data['images']
test_annotations = test_data['annotations']

In [ ]:
answer_dict = {}

for i, anno in tqdm(enumerate(test_annotations)):
    if anno['language'] == 'en':
        answer = ofa.ofa_inference(anno['img_path'], anno['question'])
        split_ans = answer.split()
        ans = []
        for w in split_ans:
            try:
                ans.append(num2words(w))
            except:
                ans.append(w)
        answer = ' '.join(ans)
        answer_dict[anno['id']] = f"{anno['question']} {answer}"
    elif anno['language'] == 'vi':
        en_question = gg_translator.translate(anno['question'], src='vi', dest='en').text
        en_answer = ofa.ofa_inference(anno['img_path'], en_question)
        vi_answer = gg_translator.translate(en_answer, src='en', dest='vi').text
        answer_dict[anno['id']] = f"{anno['question']} {vi_answer}"
#         en_question = envi_translator.translate_vi2en(anno['question'])
#         en_answer = ofa.ofa_inference(anno['img_path'], en_question)
#         vi_answer = envi_translator.translate_en2vi(en_answer)
#         answer_dict[anno['id']] = f"{anno['question']} {vi_answer}"
        
#         answer_dict[anno['id']] = ''
    else:
        en_question = gg_translator.translate(anno['question'], src='ja', dest='en').text
        en_answer = ofa.ofa_inference(anno['img_path'], en_question)
        ja_answer = gg_translator.translate(en_answer, src='en', dest='ja').text
        answer_dict[anno['id']] = f"{anno['question']} {ja_answer}"

0it [00:00, ?it/s]

In [36]:
with open('./outputs/results.json', 'w', encoding='utf-8') as f:
    json.dump(answer_dict, f, indent=4, ensure_ascii=False)

In [16]:
assert len(test_annotations) == len(answer_dict)

In [19]:
i = 0 
for k, v in answer_dict.items():
    answer_dict[k] = v.replace(test_annotations[i]['question'], '').strip()
    i += 1

In [20]:
answer_dict

{0: 'one',
 1: 'a store',
 2: '- Không.',
 3: '- Không.',
 4: '',
 5: '',
 6: 'black',
 7: 'left',
 8: 'three',
 9: 'Phố cổ Hội An',
 10: '- Không.',
 11: '- Không, không.',
 12: '',
 13: '',
 14: '',
 15: 'one',
 16: 'four',
 17: 'reading',
 18: 'motorcycles',
 19: 'one',
 20: 'một cảnh đường phố ở Hà Nội, Việt Nam',
 21: '- Vâng.',
 22: '',
 23: '',
 24: 'right',
 25: 'fruit',
 26: 'jeans',
 27: '- Không, không.',
 28: '- Không, không.',
 29: '- Không, không.',
 30: '- Không, không.',
 31: '- Không, không.',
 32: '- Không.',
 33: '',
 34: '',
 35: '',
 36: '2',
 37: 'boy',
 38: '2',
 39: 'boy',
 40: 'one',
 41: '- Vâng.',
 42: '- Vâng.',
 43: '- Không.',
 44: '- Không, không.',
 45: '- Không, không.',
 46: '',
 47: '',
 48: '',
 49: '',
 50: '',
 51: '2',
 52: 'one',
 53: 'writing',
 54: '1',
 55: '- Không.',
 56: '- Không.',
 57: '- Không, không.',
 58: '- Không.',
 59: '',
 60: '',
 61: '',
 62: '',
 63: 'green',
 64: '- Không.',
 65: '',
 66: 'market',
 67: 'shopping',
 68: 'three

In [21]:
with open('./outputs/results.json', 'w', encoding='utf-8') as f:
    json.dump(answer_dict, f, indent=4, ensure_ascii=False)

In [25]:
lang_dict = {'vi': 0, 'ja': 0, 'en': 0}
for anno in test_annotations:
    lang_dict[anno['language']] += 1

lang_dict

{'vi': 1678, 'ja': 1651, 'en': 1686}

In [32]:
num2words(5)

'five'

In [38]:
import visen
from vncorenlp import VnCoreNLP
from unicodedata import normalize as unl

In [39]:
annotator = VnCoreNLP(
    '/home/phucpx/vinbdi/MRC-for-OE-ABSA/models/pretrained/VnCoreNLP/VnCoreNLP-1.1.1.jar', 
    annotators="wseg", 
    max_heap_size='-Xmx2g'
)

def normalize(text):
    text = unl('NFKC', text)
    text = visen.clean_tone(text)
    
    seg_text = annotator.tokenize(text)
    listwords = []
    for sent in seg_text:
        for word in sent:
            listwords.append(word.replace('_', ' '))
    
    text = ' '.join(listwords)
    
    return text

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
with open('./outputs/results.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
data

{'0': 'how many people are selling pho? one',
 '1': 'what are the people standing in line for? a store',
 '2': '',
 '3': '',
 '4': '',
 '5': '',
 '6': 'what color is her handbag? black',
 '7': 'on which side of her is the bicycle parked? left',
 '8': 'how many lanterns are hung on the column? three',
 '9': '',
 '10': '',
 '11': '',
 '12': '',
 '13': '',
 '14': '',
 '15': 'how many people are sitting inside the store? one',
 '16': 'how many chairs are prepared outside the store? four',
 '17': 'what is the woman sitting inside the electrical shop doing? reading',
 '18': 'what vehicles are prohibited on this road? motorcycles',
 '19': 'how many cars are present on this street corner? one',
 '20': '',
 '21': '',
 '22': '',
 '23': '',
 '24': 'which side is the man in white shirt looking at? right',
 '25': 'what are the bicycles on the left carrying? fruit',
 '26': 'what does this man wear? jeans',
 '27': '',
 '28': '',
 '29': '',
 '30': '',
 '31': '',
 '32': '',
 '33': '',
 '34': '',
 '35':

In [41]:
for k, v in data.items():
    data[k] = normalize(v)

data

{'0': 'how many people are selling pho ? one',
 '1': 'what are the people standing in line for ? a store',
 '2': '',
 '3': '',
 '4': '',
 '5': '',
 '6': 'what color is her handbag ? black',
 '7': 'on which side of her is the bicycle parked ? left',
 '8': 'how many lanterns are hung on the column ? three',
 '9': '',
 '10': '',
 '11': '',
 '12': '',
 '13': '',
 '14': '',
 '15': 'how many people are sitting inside the store ? one',
 '16': 'how many chairs are prepared outside the store ? four',
 '17': 'what is the woman sitting inside the electrical shop doing ? reading',
 '18': 'what vehicles are prohibited on this road ? motorcycles',
 '19': 'how many cars are present on this street corner ? one',
 '20': '',
 '21': '',
 '22': '',
 '23': '',
 '24': 'which side is the man in white shirt looking at ? right',
 '25': 'what are the bicycles on the left carrying ? fruit',
 '26': 'what does this man wear ? jeans',
 '27': '',
 '28': '',
 '29': '',
 '30': '',
 '31': '',
 '32': '',
 '33': '',
 '34

In [42]:
with open('./outputs/results.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)